In [1]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import dask.dataframe as dd
import utils

class DataSequence(keras.utils.Sequence):
  
    def __init__(self, file, batch_size):
        self.file = file
        self.batch_size = batch_size
        self.ddf = dd.read_csv(self.file)
        self.total_length = len(self.ddf)
      
    def __len__(self):
        #total_length = sum(1 for row in open(self.file))
        total_length = 4000000
        return int(np.ceil(total_length) / self.batch_size)
    
    
    def __getitem__(self, idx):
        df = pd.read_csv(self.file, skiprows=idx*self.batch_size, nrows=self.batch_size)
      
        x = np.ndarray(shape=(self.batch_size,13,8,8))
        y = np.array(df.iloc[:,1])
      
        #have preprocessing here right now for testing, should save processed scores in file
        # once we decide how to do this
        for i, f in enumerate(df.iloc[:,0]):
            x[i] = utils.vectorize(f)
        x = x.reshape((self.batch_size,832))
        y = utils.preprocess_scores(y)
        #print(x.shape, y.shape)
        #print(idx)
        
        return (x, y)

In [ ]:
model = tf.keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=(13, 8, 8)),
    tf.keras.layers.Dense(832, activation='linear'),
    tf.keras.layers.Dense(832, activation='relu'),
    tf.keras.layers.Dense(832, activation='linear'),
    tf.keras.layers.Dense(832, activation='relu'),
    tf.keras.layers.Dense(832, activation='linear'),
    tf.keras.layers.Dense(832, activation='relu'),
    #tf.keras.layers.Dense(1, activation=scaled_sigmoid)
    tf.keras.layers.Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam',
              loss='MeanAbsoluteError')

train_sequence = DataSequence("../data/kaggleDataset/chessData.csv", 1024)

# Train the model
model.fit(train_sequence, epochs=10)

model.save('saved_model')

In [2]:
ddf = dd.read_csv("../data/createdData/preprocessedDataset.csv")

In [14]:
df = ddf.loc[0:2]
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,Evaluation
npartitions=337,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [7]:
pd.read_csv("../data/createdData/preprocessedDataset.csv", nrows=10_000, dtype=np.int8)

,0,1,2,3,4,5,6,7,8,9,...,823,824,825,826,827,828,829,830,831,Evaluation
0,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-10
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,15
2,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-9
3,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,15
4,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-15
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
9997,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-15
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14
